In [30]:
%pip install -qU langchain langchain_ollama faiss-cpu arize-phoenix openinference.instrumentation.langchain

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
import phoenix as px


In [2]:
os.environ["PHOENIX_TRACING"] = "true"
px.launch_app()


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [3]:

import nest_asyncio
from langchain.chains import RetrievalQA
from openinference.instrumentation.langchain import LangChainInstrumentor

from phoenix.otel import register

nest_asyncio.apply()  # needed for concurrent evals in notebook environments

In [4]:
tracer_provider = register()
LangChainInstrumentor(tracer_provider=tracer_provider).instrument(skip_dep_check=False)

OpenTelemetry Tracing Details
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [5]:
loader = TextLoader("../00-example_data/state_of_the_union.txt")  # Replace with your document path
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)


In [6]:
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever()


In [7]:
llm = ChatOllama(model="llama3.2:latest")

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant who answers questions based on retrieved documents."),
    ("human", "{question}")
])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Combines retrieved documents into context
    retriever=retriever
)


In [8]:
query = "What did the president say about climate change?"
response = qa_chain.invoke(query)

print("📢 Question:", query)
print("✅ Answer:", response["result"])


📢 Question: What did the president say about climate change?
✅ Answer: The president mentioned combating climate change as one of the key initiatives to provide investments and tax credits to weatherize homes and businesses, cutting energy costs for families by an average of $500 per year. They also aimed to double America's clean energy production in solar, wind, and other sources, lower the price of electric vehicles, and promote environmental justice to withstand the devastating effects of the climate crisis.
